# Part C:

Let's first import some required libraries:

In [1]:
import numpy as np
import scipy.linalg as la
from scipy import sparse as sps
from scipy.sparse import linalg as spla

### Inverse method:

In [2]:
def inv_method(X, y, w):
    '''
   Solving WLS via inversion method
    
    Inputs:
    X: An N x P array of features
    y: An N-array of responses
    w: An N-array of weights
    
    Outputs:
    Beta, A P-array solution vector for weighted least squares
    '''
    X_TW = X.T * w
    X_TWXinv = la.inv(np.dot(X_TW, X))
    X_TWy = np.dot(X_TW, y)
    Beta = np.dot(X_TWXinv, X_TWy)
    return Beta

### QR Factorization

In [3]:
def QR(X, y, w):
    '''
    Solve WLS using QR factorization
    
    Inputs:
    X: An N x P array of features
    y: An N-array of responses
    w: An N-array of weights
    
    Outputs:
    Beta, A P-array solution vector for weighted least squares
    '''
    
    # Take W^.5X (note that this is a simple broadcast)
    w_5X = (w**.5).reshape(w.shape[0],1) * X
    # Compute the reduced QR factorization of W^.5 X
    Q, R = la.qr(w_5X, mode='economic')
    # Compute Q^T W^.5 y
    QTW_5y = np.dot(Q.T * w**.5,y)
    # Solve the upper triangular system
    beta = la.solve_triangular(R, QTW_5y)
    return beta

### Cholesky Factorization

The Cholesky decomposition is often used as a fast way of solving

$$A \mathbf{x} = \mathbf{b}$$

(when A is both Hermitian/symmetric and positive-definite).

First, we solve for \mathbf{y} in

$$L \mathbf{y} = \mathbf{b},$$

and then for $\mathbf{x}$ in

$$L.H \mathbf{x} = \mathbf{y}.$$



In [4]:
def Cholesky(X, y, w):
    '''
    Solve WLS using Cholesky factorization
    
    Inputs:
    X: An N x P array of features
    y: An N-array of responses
    w: An N-array of weights
    
    Outputs:
    Beta, A P-array solution vector for weighted least squares
    '''
    X_TW = X.T * w
    X_TWX = np.dot(X_TW, X)

    X_TWy = np.dot(X_TW, y)
    L = la.cho_factor(X_TWX, lower=False, overwrite_a=False, check_finite=True)
    Beta = la.cho_solve(L, X_TWy, overwrite_b=False, check_finite=True)
    
    return Beta

In [5]:
# Benchmarking for part C:

N = 50000
P = 1000
X = np.random.rand(N, P) 
y = np.random.rand(N) 
W = np.ones(N)

In [6]:
%%timeit 
'''
Cholesky Factorization method timing
'''   
B_1 = Cholesky(X,y, W)

1 loop, best of 3: 2.16 s per loop


In [7]:
%%timeit 
'''
QR Factorization method timing
'''   
B_1 = QR(X,y, W)

1 loop, best of 3: 13.5 s per loop


As we can see, the QR Factorization take more time than the both Cholesky and inversion methods, however this method is much more stable.

In [8]:
%%timeit
"""
Inversion method timing
"""
B_2 = inv_method(X,y,W)

1 loop, best of 3: 2.31 s per loop


# Part D:

In [9]:
def sparse_solver(X, y, w):
    '''
    Solves WLS by considering the sparsity of X.
    
    Inputs:
    X: An N x P sparse matrix (from scipy.sparse). 
    y: An N array of responses
    w: An N array of weights
    
    Outputs
    Beta: A dense P array that is the solution vector to the sparse system.
    '''
    
    # Begin by making sure that X is in csr format, which allows for matrix multiplication
    if X.format != 'csr':
        X = X.asformat('csr')
        
    # Set W to be a sparse diagonal matrix with values w
    W = sps.diags(w, format='csr')
    # Compute X_TW using the csr dot method.
    # Note that it might be better to use some kind of sparse broadcast, but I have no idea how
    X_TW = X.transpose().dot(W)
    # Again, use the sparse csr dot method to compute the linear algebra solver
    X_TWX = X_TW.dot(X)
    X_TWy = X_TW.dot(y)
    Beta = spla.spsolve(X_TWX, X_TWy)
    return Beta

In [10]:
# Benchmarking part D
N = 50000
P = 1000
X = sps.random(N, P)
dense_X = X.toarray()
w = np.random.rand(N)
y = np.random.rand(N)

In [11]:
%%timeit 
Cholesky(dense_X,y, w) # Benchmark QR

1 loop, best of 3: 2.3 s per loop


In [12]:
%%timeit 
QR(dense_X,y, w) # Benchmark QR

1 loop, best of 3: 13.5 s per loop


In [13]:
%%timeit
inv_method(dense_X,y,w) # Benchmark Inverse method

1 loop, best of 3: 1.94 s per loop


In [14]:
%%timeit
sparse_solver(X,y,w) # Benchmark sparse method

1 loop, best of 3: 278 ms per loop


**NOTE:** As we can see, the sparse_solver is faster than the other methods:

In [16]:
# Benchmarking for different sparsity levels:
for d in [0.01, 0.05, 0.1, 0.2, 0.5, 1.0]:
    print('-----------sparsity levels =', d, '-----------')
    N = 50000
    P = 1000
    X = sps.random(N, P, density=d)
    dense_X = X.toarray()
    w = np.random.rand(N)
    y = np.random.rand(N)
    print('Cholesky Method:')
    %time B0 = Cholesky(dense_X,y,w)
    print('\n')
    print('QR Decomposition method:')
    %time B1 = QR(dense_X,y,w)
    print('\n')
    print('Inverse method:')
    %time B2 = inv_method(dense_X,y,w)
    print('\n')
    print('sparse solver method:')
    %time B3 = sparse_solver(X,y,w)
    print("\n")

-----------sparsity levels = 0.01 -----------
Cholesky Method:
CPU times: user 3.05 s, sys: 483 ms, total: 3.53 s
Wall time: 2.16 s


QR Decomposition method:
CPU times: user 21.8 s, sys: 1.15 s, total: 22.9 s
Wall time: 13.9 s


Inverse method:
CPU times: user 3.81 s, sys: 249 ms, total: 4.06 s
Wall time: 2.45 s


sparse solver method:
CPU times: user 704 ms, sys: 68.5 ms, total: 772 ms
Wall time: 583 ms


-----------sparsity levels = 0.05 -----------
Cholesky Method:
CPU times: user 3.01 s, sys: 159 ms, total: 3.17 s
Wall time: 1.71 s


QR Decomposition method:
CPU times: user 21.9 s, sys: 1.33 s, total: 23.2 s
Wall time: 13.8 s


Inverse method:
CPU times: user 3.85 s, sys: 269 ms, total: 4.12 s
Wall time: 2.21 s


sparse solver method:
CPU times: user 2.57 s, sys: 123 ms, total: 2.7 s
Wall time: 2.43 s


-----------sparsity levels = 0.1 -----------
Cholesky Method:
CPU times: user 3.83 s, sys: 217 ms, total: 4.04 s
Wall time: 2.25 s


QR Decomposition method:
CPU times: user 25.3 s